# Install and Import Packages

In [ ]:
!pip install transformers
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
# The same as Torch version and CUDA version (torch.__version__ is 2.0.0+cu118)
!pip install torch-geometric
!python3 -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-06 01:34:50.126366: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
print(torch.__version__)
from torch_geometric.nn import GCNConv, global_mean_pool, global_max_pool
from torch_geometric.data import Data, DataLoader
from transformers import BertModel, BertTokenizer, BertConfig
import csv
#import spacy
import networkx as nx
#from gensim.models import Word2Vec
from tqdm import tqdm
import xml.etree.ElementTree as ET
#from nltk.corpus import wordnet as wn
#from nltk.corpus.reader.wordnet import Synset
#from nltk.wsd import lesk
#import nltk
import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from google.colab import drive
drive.mount('/content/drive')

#nltk.download('wordnet')
#nltk.download('punkt')

2.0.0+cu118
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#nlp = spacy.load('en_core_web_sm') # DO NOT split a sentence for more 1 time
# If spliting a sentence more than 1 time, the result may different from the same split
model_path = "/content/drive/My Drive/my_GNN_MTL_PT_model_v3.pt"  # Choose your desired path and filename
# Model parameters
bert_model_name = 'bert-base-uncased'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Custom GNN and BiLSTM layers
class GNN(torch.nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        # input_dim is number of features of each node
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_weight, batch):
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)

        # Global Pooling (stack different aggregations)
        hidden = torch.cat([global_mean_pool(x, batch),
                            global_max_pool(x, batch)], dim=1)
        return x, hidden


class BERT_GNN_MTL_Classifier(nn.Module):
    def __init__(self, num_classes, hidden_dim, num_lstm_layers, gnn_hidden_dim, gnn_output_dim, dropout, max_length, batch_size):
        super().__init__()
        self.max_length = max_length
        self.batch_size = batch_size
        self.gnn_output_dim = gnn_output_dim
        self.bert_config = BertConfig.from_pretrained(bert_model_name)
        self.bert = BertModel.from_pretrained(bert_model_name, config=self.bert_config)
        # Embedding size of BERT is 768
        self.gnn = GNN(self.bert_config.hidden_size, gnn_hidden_dim, gnn_output_dim)
        #self.bilstm = BiLSTM(self.bert_config.hidden_size, hidden_dim, num_lstm_layers)
        #self.bilstm = BiLSTM(gnn_output_dim, hidden_dim, num_lstm_layers)
        self.bilstm = nn.LSTM(
            input_size=gnn_output_dim,
            hidden_size=hidden_dim,
            num_layers=num_lstm_layers,
            bidirectional=True,
            batch_first=True,
            #dropout=dropout if num_layers > 1 else 0
        )
        self.bilstm_2 = nn.LSTM(
            input_size=self.bert.config.hidden_size,
            hidden_size=hidden_dim,
            num_layers=num_lstm_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_lstm_layers > 1 else 0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, 1) # 2 is bidirectional LSTM, has 2, 1 is the output
        # self.fc = nn.Linear(hidden_dim * 2, num_classes) # multi-class classfication
        # self.sigmoid = nn.Sigmoid()
        #self.classifier = nn.Linear(hidden_dim * (2 if self.bilstm.bidirectional else 1), 1) # sigmoid
        self.fc2 = nn.Linear(gnn_output_dim * 2 + hidden_dim * 2, 256)
        self.classifier2 = nn.Linear(hidden_dim * (2 if self.bilstm.bidirectional else 1) + 256, 1) # sigmoid

    def forward(self, data, token_index):
        # token_index[i] should be start with 0 and step is 1, because the GNN use only one id per word
        input_ids, attention_mask = data.input_ids, data.attention_mask
        corresponding_index, sentence_len = data.corresponding, data.sentence_len
        edge_index, edge_weight, num_nodes, batch = data.edge_index, data.edge_weight, data.num_nodes, data.batch
        #print(sentence_len)
        assert sum(sentence_len) == len(corresponding_index)
        actual_batch_size = len(input_ids) // self.max_length

        first_cut = 0
        corres = [] # the corresponding index of each sentence
        # print(corresponding_index)
        # print(sentence_len)
        for each_sent_len_i in range(len(sentence_len) - 1):
            corres.append(corresponding_index[first_cut:first_cut+sentence_len[each_sent_len_i]])
            first_cut += sentence_len[each_sent_len_i]
        corres.append(corresponding_index[first_cut:first_cut+sentence_len[-1]])
        # print(corres)
        assert len(corres) == actual_batch_size

        input_ids = torch.reshape(input_ids, (actual_batch_size, self.max_length))
        attention_mask = torch.reshape(attention_mask, (actual_batch_size, self.max_length))
        # print("input_ids.shape:", input_ids.shape) # torch.Size([self.batch_size, self.max_length])

        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        hidden_states = bert_output.last_hidden_state
        # print("hidden_states.shape:", hidden_states.shape) # torch.Size([self.batch_size, self.max_length, 768])

        gnn_input = torch.zeros((len(corresponding_index), hidden_states.shape[-1])) # not include the start and end ID
        each_batch_index = 0
        start_index = 0
        each_counter = 0
        # print(corresponding_index)
        for each_corr_i in corresponding_index:
            # each_corr_i + 1 because the input_ids has a start ID in the front,
            # which needs to add 1 to match the index
            gnn_input[start_index] = hidden_states[each_batch_index, each_corr_i + 1]
            start_index += 1
            each_counter += 1
            if each_counter == sentence_len[each_batch_index]:
                each_batch_index += 1
                each_counter = 0
        each_counter = 0
        assert each_batch_index == actual_batch_size
        assert start_index == len(corresponding_index)
        
        gnn_output, gnn_hidden = self.gnn(x=gnn_input, edge_index=edge_index, edge_weight=edge_weight, batch=batch)
        #print(gnn_hidden.shape)

        bilstm_input = torch.zeros((actual_batch_size, self.max_length, self.gnn_output_dim))
        each_batch_index = 0
        start_index = 0
        # Similar to padding 0
        for each_sen_len in sentence_len:
            for each_word_i in range(each_sen_len):
                bilstm_input[each_batch_index, each_word_i] = gnn_output[start_index]
                start_index += 1
            each_batch_index += 1

        each_batch_index = 0
        start_index = 0

        lstm_output, (hidden, _) = self.bilstm(bilstm_input)
        '''
        Anthor way is using only last hidden state of the LSTM cell:

        Using lstm_output[:, -1] selects the last hidden state of the LSTM cell
        for each sequence in the batch. The reason we use this approach in the
        example provided is that the last hidden state is often a good
        representation of the entire sequence in many sequence-to-sequence
        models, especially for classification tasks.

        When we use lstm_output[:, -1], we're selecting the hidden states of the
        LSTM cells at the last time step (i.e., the last token in the input sequence)
        for each sequence in the batch. This can be a good representation of the
        entire sequence for classification tasks since it captures information
        from both the forward and backward passes of the sequence.
        '''
        pooled_output = torch.mean(lstm_output, 1)

        if self.bilstm.bidirectional:
            hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
            # hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=-1)
        else:
            hidden = hidden[-1, :, :]

        #print(hidden.shape)
        hidden_gnn_bilstm = torch.cat([gnn_hidden, hidden], dim=-1)
        #print(hidden_gnn_bilstm.shape)
        connected_layer = self.fc2(self.dropout(hidden_gnn_bilstm))
        #print("connected_layer.shape", connected_layer.shape)

        # Get the hidden state of the word at the specified index
        assert lstm_output.size(0) == len(input_ids)
        assert actual_batch_size == lstm_output.size(0)
        assert lstm_output.size(2) == self.bilstm.hidden_size * (2 if self.bilstm.bidirectional else 1)

        lstm_output_2, (hidden_2, _) = self.bilstm_2(hidden_states)

        assert lstm_output_2.size(2) == self.bilstm_2.hidden_size * (2 if self.bilstm_2.bidirectional else 1)

        focused_word_hidden = torch.zeros((lstm_output_2.size(0), lstm_output_2.size(2)))
        for sentence_index in range(lstm_output_2.size(0)):
            focused_word_hidden[sentence_index] = lstm_output_2[sentence_index, token_index[sentence_index], :]
        #classification_output = self.classifier(focused_word_hidden)
        classification_output = self.classifier2(torch.cat([focused_word_hidden, connected_layer], dim=-1))

        return self.fc(pooled_output), classification_output
        

In [ ]:
# **sigmoid**
# If using sigmoid, we should use this accuracy function
def calculate_accuracy(output, target):
    assert output.shape == target.shape
    threshold = 0.5
    predictions = (output > threshold).float()
    correct = (predictions == target).sum().item()
    total = target.numel()
    return correct / total

def train_epoch(model, data_loader, loss_fn_1, loss_fn_2, optimizer, device):
    model.train()
    total_loss = 0.0
    total_accuracy_1 = 0.0
    total_accuracy_2 = 0.0
    for batch in tqdm(data_loader):
        batch = batch.to(device)
        labels = batch.y.unsqueeze(1).float().to(device)

        corresponding_token_index = torch.clone(batch.corresponding_location).to(device)

        batch_att = torch.reshape(batch.attention_mask, (len(labels), len(batch.attention_mask) // len(labels)))

        for sentence_index in range(len(labels)):
            if corresponding_token_index[sentence_index] < 0:
                # No pun word
                assert labels[sentence_index][0] < 0.5
                corresponding_token_index[sentence_index] = torch.randint(1, int(sum(batch_att[sentence_index])) - 1, (1,))
            else:
                corresponding_token_index[sentence_index] += 1 # the BERT has start ID
        # print(corresponding_token_index)
        # print(batch.id_range)

        optimizer.zero_grad()
        logits_1, logits_2 = model(batch, corresponding_token_index)
        loss_1 = loss_fn_1(logits_1, labels)
        loss_2 = loss_fn_2(logits_2, labels)
        joint_loss = loss_1 + loss_2
        joint_loss.backward()
        optimizer.step()

        total_loss += joint_loss.item()
        sigmoid_1 = torch.sigmoid(logits_1.view(-1)).unsqueeze(1)
        sigmoid_2 = torch.sigmoid(logits_2.view(-1)).unsqueeze(1)
        total_accuracy_1 += calculate_accuracy(sigmoid_1, labels)
        total_accuracy_2 += calculate_accuracy(sigmoid_2, labels)
        #print(f"Train Loss: {total_loss:.4f}, Train Accuracy: {total_accuracy:.4f}")

    return total_loss / len(data_loader), total_accuracy_1 / len(data_loader), total_accuracy_2 / len(data_loader)


def eval_epoch(model, data_loader, loss_fn_1, loss_fn_2, device):
    model.eval()
    total_loss = 0.0
    total_accuracy_1 = 0.0
    total_accuracy_2 = 0.0
    with torch.no_grad():
        for batch in tqdm(data_loader):
            batch = batch.to(device)
            labels = batch.y.unsqueeze(1).float().to(device)

            corresponding_token_index = torch.clone(batch.corresponding_location).to(device)

            batch_att = torch.reshape(batch.attention_mask, (len(labels), len(batch.attention_mask) // len(labels)))

            for sentence_index in range(len(labels)):
                if corresponding_token_index[sentence_index] < 0:
                    # No pun word
                    assert labels[sentence_index][0] < 0.5
                    corresponding_token_index[sentence_index] = torch.randint(1, int(sum(batch_att[sentence_index])) - 1, (1,))
                    # random select one location
                else:
                    corresponding_token_index[sentence_index] += 1  # the BERT has start ID

            logits_1, logits_2 = model(batch, corresponding_token_index)
            loss_1 = loss_fn_1(logits_1, labels)
            loss_2 = loss_fn_2(logits_2, labels)
            joint_loss = loss_1 + loss_2

            total_loss += joint_loss.item()
            sigmoid_1 = torch.sigmoid(logits_1.view(-1)).unsqueeze(1)
            sigmoid_2 = torch.sigmoid(logits_2.view(-1)).unsqueeze(1)
            total_accuracy_1 += calculate_accuracy(sigmoid_1, labels)
            total_accuracy_2 += calculate_accuracy(sigmoid_2, labels)
            #print(f"Validation Loss: {total_loss:.4f}, Validation Accuracy: {total_accuracy:.4f}")

    return total_loss / len(data_loader), total_accuracy_1 / len(data_loader), total_accuracy_2 / len(data_loader)

In [ ]:
# Assuming you have your data as lists: train_texts, train_labels, val_texts, and val_labels
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
max_length = 80  # Adjust the maximum length based on your dataset
batch_size = 32

In [ ]:
fileObj = open('/content/drive/My Drive/train_dataset.obj', 'rb')
train_dataset = pickle.load(fileObj)
fileObj.close()

fileObj = open('/content/drive/My Drive/val_dataset.obj', 'rb')
val_dataset = pickle.load(fileObj)
fileObj.close()

# Create DataLoaders for each set with a batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Initialize the custom BERT model
# num_classes = len(set(total_gold))  # Assuming labels are integers starting from 0
num_classes = 2
hidden_dim = 128
num_lstm_layers = 2
gnn_hidden_dim = 512
gnn_output_dim = 256
learning_rate = 2e-5
dropout = 0.3

# Set up two different loss function
loss_fn_1 = nn.BCEWithLogitsLoss()
loss_fn_2 = nn.BCEWithLogitsLoss()

In [ ]:
# Build the model
model = BERT_GNN_MTL_Classifier(num_classes, hidden_dim, num_lstm_layers,
                                gnn_hidden_dim, gnn_output_dim, dropout,
                                max_length, batch_size).to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT_GNN_MTL_Classifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [ ]:
# Train the model
num_epochs = 5

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss, train_acc_1, train_acc_2 = train_epoch(model, train_loader, loss_fn_1, loss_fn_2, optimizer, device)
    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc_1:.4f}, Training Word Accuracy: {train_acc_2:.4f}")
    torch.save(model, model_path[:-3] + '_epoch_' + str(epoch) + '.pt') # save the entire model, including the architecture
    val_loss, val_acc_1, val_acc_2 = eval_epoch(model, val_loader, loss_fn_1, loss_fn_2, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc_1:.4f}, Validation Word Accuracy: {val_acc_2:.4f}")
print("Training complete.")

Epoch 1/5


100%|██████████| 144/144 [1:05:36<00:00, 27.34s/it]


Training Loss: 0.9755, Training Accuracy: 0.7357, Training Word Accuracy: 0.8622


100%|██████████| 36/36 [04:59<00:00,  8.32s/it]


Validation Loss: 0.6974, Validation Accuracy: 0.9382, Validation Word Accuracy: 0.9374
Epoch 2/5


...

100%|██████████| 144/144 [1:02:18<00:00, 25.96s/it]


Training Loss: 0.1810, Training Accuracy: 0.9748, Training Word Accuracy: 0.9770


100%|██████████| 36/36 [04:56<00:00,  8.25s/it]


Validation Loss: 0.4566, Validation Accuracy: 0.9313, Validation Word Accuracy: 0.9313
Epoch 5/5


 49%|████▊     | 70/144 [30:37<32:24, 26.27s/it]

In [ ]:
# Load the the saved file
loaded_model = torch.load("/content/drive/My Drive/my_GNN_MTL_PT_model_v3_epoch_4.pt").to(device)

# Set the model to evaluation mode if you plan to use it for inference
loaded_model.eval()

location_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

with torch.no_grad():

    y_true_hom = []
    y_pred_hom = []
    y_true_het = []
    y_pred_het = []
    y_true_neg = []
    y_pred_neg = []

    location_hom_correct = 0
    location_het_correct = 0

    num_correct, num_samples, num_accuracy = 0, 0, 0
    for ele in tqdm(location_loader):
        # print(ele.attention_mask)
        attention_mask = ele.attention_mask.reshape(1, max_length).to(device)
        #labels = ele['label'].unsqueeze(1).float().to(device)
        label = ele.y.to(device)
        # corresponding_token_index = torch.clone(ele.corresponding_location).to(device)
        token_index = torch.clone(ele.location).to(device) # After GNN, the same as original, start with 0
        id_range = ele.id_range.to(device)
        data_identification = ele.data_identification.to(device)

        #print(input_ids)
        #print(attention_mask)
        #print(label[0])
        #print(int(token_index[0]))
        #print(id_range)
        #print(ele)

        if float(label[0]) < 0.5:
            y_true_neg.append(0)
            # This has not pun, do not count
            logits_1, _ = loaded_model(ele, torch.tensor(0).reshape(1,))
            y_pred_neg.append(0 if float(torch.sigmoid(logits_1.view(-1))) < 0.5 else 1)
            continue
        
        scores_list = torch.zeros((int(sum(attention_mask[0])) - 2,))
        for used_token_index in range(1, int(sum(attention_mask[0])) - 1):
            # After GNN, the index is same as original, start with 0
            logits_1, logits_2 = loaded_model(ele, torch.tensor(used_token_index).reshape(1,))
            scores_list[used_token_index - 1] = float(torch.sigmoid(logits_2.view(-1))) # Get each ID scores

        # If the matched token ID is belonged to the pun word, it is also correct.
        if int(id_range[int(token_index[0])][0]) <= int(torch.argmax(scores_list, dim=0)) < int(id_range[int(token_index[0])][1]):
            num_correct += 1

            if int(data_identification[0]) < 2250:
                location_hom_correct += 1
            elif 2250 <= int(data_identification[0]) < 2250 + 1780:
                location_het_correct += 1
            else:
                print("Something Wrong")
        
        if int(data_identification[0]) < 2250:
            y_true_hom.append(1)
            y_pred_hom.append(0 if float(torch.sigmoid(logits_1.view(-1))) < 0.5 else 1)
        elif 2250 <= int(data_identification[0]) < 2250 + 1780:
            y_true_het.append(1)
            y_pred_het.append(0 if float(torch.sigmoid(logits_1.view(-1))) < 0.5 else 1)
        else:
            print("Something Wrong")

        num_samples += 1
        #if num_samples == 300:
        #    break

    num_accuracy = num_correct / num_samples
    print(f'Location Accuracy: {num_accuracy:.4f}')
    print(f'Homographic Location Accuracy: {(location_hom_correct / len(y_true_hom)):.4f}')
    print(f'Heterographic Location Accuracy: {(location_het_correct / len(y_true_het)):.4f}')
    assert len(y_true_hom) + len(y_true_het) == num_samples
    print()
    print(classification_report(y_true_hom+y_true_neg[:len(y_true_hom)], y_pred_hom+y_pred_neg[:len(y_true_hom)], target_names=['No Puns', 'Homographic']))
    print(confusion_matrix(y_true_hom+y_true_neg[:len(y_true_hom)], y_pred_hom+y_pred_neg[:len(y_true_hom)]))
    print()
    print(classification_report(y_true_het+y_true_neg[:len(y_true_het)], y_pred_het+y_pred_neg[:len(y_true_het)], target_names=['No Puns', 'Heterographic']))
    print(confusion_matrix(y_true_het+y_true_neg[:len(y_true_het)], y_pred_het+y_pred_neg[:len(y_true_het)]))

100%|██████████| 1151/1151 [55:06<00:00,  2.87s/it]

Location Accuracy: 0.5115
Homographic Location Accuracy: 0.4822
Heterographic Location Accuracy: 0.5465

              precision    recall  f1-score   support

     No Puns       0.97      0.93      0.95       309
 Homographic       0.93      0.97      0.95       309

    accuracy                           0.95       618
   macro avg       0.95      0.95      0.95       618
weighted avg       0.95      0.95      0.95       618

[[286  23]
 [  8 301]]

               precision    recall  f1-score   support

      No Puns       0.95      0.92      0.93       258
Heterographic       0.92      0.95      0.94       258

     accuracy                           0.93       516
    macro avg       0.93      0.93      0.93       516
 weighted avg       0.93      0.93      0.93       516

[[237  21]
 [ 13 245]]
